In [1]:
import pandas as pd  # module for working with data sets (need for working inner classes and modules)
import traceback
import datetime
import warnings
import numpy as np
import os  # module for working with operating system catalog structure
import openpyxl  # module for working with Excel files
import time  # module for working with date and time
import pyodbc  # module for working with databases
import win32com.client  # Module for generating MS access data base

from datetime import datetime  # Module for working with date and time data
from win32com.client import Dispatch  # Module for generating MS access data base
from tkinter.filedialog import askopenfilename  # Module for open file with win gui

# show an "Open" dialog box and return the path to the selected file
filename_comp = askopenfilename(title="Select file for compare", filetypes=[("excel files", "*.xlsx")])
filename_new = askopenfilename(title="Select new file", filetypes=[("excel files", "*.xlsx")])

# Ignoring pandas version errors
warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

#  Columns with necessary information
inf_columns = ['Наименование объекта/комплекта РД',
               'Коды работ по выпуску РД', 
               'Пакет РД', 
               'Код KKS документа',
               'Статус Заказчика', 
               'Текущая ревизия', 
               'Статус текущей ревизии',
               'Дата выпуска РД по договору подрядчика',
               'Дата выпуска РД по графику с Заказчиком',
               'Дата статуса Заказчика',
               'Ожидаемая дата выдачи РД в производство', 
               'Письма',
               'Источник информации', 
               'Разработчики РД (актуальные)', 
               'Статус РД в 1С'
               ]

#  Use columns numbers for next actions
col_numb = len(inf_columns)

# read Excel files with current and new data
print('Read excel files with current and new data')
base_df = pd.read_excel(filename_comp)[inf_columns]
base_df_1 = pd.read_excel(filename_comp, sheet_name='блок 1')[inf_columns]
base_df_2 = pd.read_excel(filename_comp, sheet_name='блок 2')[inf_columns]
base_df_3 = pd.read_excel(filename_comp, sheet_name='блок 3')[inf_columns]
base_df_4 = pd.read_excel(filename_comp, sheet_name='блок 4')[inf_columns]
new_df = pd.read_excel(filename_new)

# Finding missed rows
print('Finding missed rows')
missed_df = pd.concat([base_df, base_df_1, base_df_1]).drop_duplicates(keep=False)
missed_df = pd.concat([missed_df, base_df_2, base_df_2]).drop_duplicates(keep=False)
missed_df = pd.concat([missed_df, base_df_3, base_df_3]).drop_duplicates(keep=False)
missed_df = pd.concat([missed_df, base_df_4, base_df_4]).drop_duplicates(keep=False)

#  Clear the data in both dataframe
print('Clear the empty rows in both dataframes')
base_df = base_df.dropna(subset=['Коды работ по выпуску РД'])
base_df['Разработчики РД (актуальные)'] = base_df['Разработчики РД (актуальные)'].apply(
    lambda row: base_df['Разработчик РД'] if row is None else row
    )

# Removing unnecessary data
print('Clear the unnecessary data in base dataframe')
base_df = base_df.loc[(base_df['Коды работ по выпуску РД'].str.contains('.C.') == False)]
base_df = base_df.loc[((base_df['Код KKS документа'].str.contains('.KZ.') == False) |
                       (base_df['Код KKS документа'].str.contains('.EK.') == False) |
                       (base_df['Код KKS документа'].str.contains('.TZ.') == False) |
                       (base_df['Код KKS документа'].str.contains('.KM.') == False) |
                       (base_df['Код KKS документа'].str.contains('.GR.') == False) 
                       )]
# base_df.count()

#  Making copy of original dataframes
print('Making copy of original dataframes')
base_df_copy = base_df.copy()
new_df_copy = new_df.copy()

#  Merging two dataframes
print('Merging two dataframes')
m_df_1 = (new_df_copy.merge(base_df_copy,
                           how='left',
                           on=['Коды работ по выпуску РД', 'Код KKS документа'],
                           suffixes=['', '_new'], 
                           indicator=True))

tmp_df = m_df_1[m_df_1['_merge'] == 'left_only'][new_df_copy.columns]
m_df_2 = (tmp_df.merge(base_df_copy,
                           how='left',
                           on=['Коды работ по выпуску РД', 'Наименование объекта/комплекта РД'],
                           suffixes=['', '_new'],
                           indicator=True
                           ))

#  Preparation columns list with necessary information
tmp = np.append(m_df_1.columns[0:20].values, m_df_1.columns[-2])
columns = np.append(m_df_2.columns[0:4], m_df_2.columns[20:32])
columns = np.append(columns, m_df_2.columns[16:20])
columns = np.append(columns, m_df_2.columns[-2])
tmp_columns = np.append(m_df_1.columns[0], m_df_1.columns[20])
tmp_columns = np.append(tmp_columns, m_df_1.columns[2:4])
tmp_columns = np.append(tmp_columns, m_df_1.columns[21])
tmp_columns = np.append(tmp_columns, m_df_1.columns[5])
tmp_columns = np.append(tmp_columns, m_df_1.columns[22:32])
tmp_columns = np.append(tmp_columns, m_df_1.columns[16:20])
tmp_columns = np.append(tmp_columns, m_df_1.columns[-2])

#  Generate temporary dataframe for next appending for changed documents
print('Initiate dataframe with changed documents')
tmp_df = m_df_1[m_df_1['_merge'] == 'both'][tmp_columns]
tmp_df.columns = tmp

# Generate result dataframe, with remove document firstly
print('Generating result dataframe, with remove document firstly')
changed_df = m_df_2[m_df_2['_merge'] == 'left_only'][tmp]
tmp_df = pd.concat([changed_df, tmp_df])

#  Generate result dataframe 
print('Generating result dataframe ')
changed_df = m_df_2[m_df_2['_merge'] == 'both'][columns]
changed_df.columns = tmp
changed_df = pd.concat([changed_df, tmp_df])

# Changing dataframe
print('Changing dataframe')
changed_df['Дата выпуска РД по договору подрядчика'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика']).dt.date
changed_df['Дата выпуска РД по графику с Заказчиком'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком']).dt.date
changed_df['Дата статуса Заказчика'] = pd.to_datetime(changed_df['Дата статуса Заказчика']).dt.date
changed_df['Ожидаемая дата выдачи РД в производство'] = changed_df['Ожидаемая дата выдачи РД в производство'].apply(
    lambda row: row if row == 'в производстве' or row == 'В производстве' else pd.to_datetime(row).date()
    )
changed_df['Объект'] = changed_df['Объект'].apply(
    lambda row: changed_df['Коды работ по выпуску РД'].str.slice(0, 5) if row is None else row
    )
changed_df['Статус текущей ревизии'] = changed_df['Статус текущей ревизии'].apply(
    lambda row: base_df['Статус РД в 1С'] if row is None else row
    )
changed_df['WBS'] = changed_df['WBS'].apply(
    lambda row: row if row is not None else changed_df['Коды работ по выпуску РД'].apply(lambda row: row[6 : row.find('.', 6)])
    )
# Формирование базы с датами
# columns = ['Система',
#            'Наименование объекта/комплекта РД',
#            'Коды работ по выпуску РД',
#            'Ожидаемая дата выдачи РД в производство',
#            'Дата выпуска РД по договору подрядчика',
#            'Дата выпуска РД по графику с Заказчиком',
#            'Ожидаемая дата выдачи РД в производство_new',
#            'Дата выпуска РД по договору подрядчика_new',
#            'Дата выпуска РД по графику с Заказчиком_new']
# new_columns = ['Система',
#                'Наименование объекта/комплекта РД',
#                'Коды работ по выпуску РД',
#                'Ожидаемая дата выдачи РД в производство_old',
#                'Дата выпуска РД по договору подрядчика_old',
#                'Дата выпуска РД по графику с Заказчиком_old',
#                'Ожидаемая дата выдачи РД в производство_new',
#                'Дата выпуска РД по договору подрядчика_new',
#                'Дата выпуска РД по графику с Заказчиком_new']
# changed_df = pd.concat([m_df_1[m_df_1['_merge'] == 'both'][columns], m_df_2[m_df_2['_merge'] == 'left_only'][columns]])
# changed_df = pd.concat([changed_df, m_df_2[m_df_2['_merge'] == 'both'][columns]])
# changed_df =  changed_df[~changed_df['Система'].isin(['ГПМ', 'ИМ', "КИТС ФЗ", "МиЗ", "ОРС", "РЗА", "Связь", "СКУ ЭЧ ОС СН", "СКУ ЭЧ ЭБ", "Технология"])]
# changed_df['Дата выпуска РД по договору подрядчика'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика']).dt.date
# changed_df['Дата выпуска РД по графику с Заказчиком'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком']).dt.date
# changed_df['Дата выпуска РД по договору подрядчика_new'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика_new']).dt.date
# changed_df['Дата выпуска РД по графику с Заказчиком_new'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком_new']).dt.date
# changed_df['Ожидаемая дата выдачи РД в производство'] = changed_df['Ожидаемая дата выдачи РД в производство'].apply(
#     lambda row: row if row == 'в производстве' or row == 'В производстве' else pd.to_datetime(row).date()
#     )
# changed_df['Ожидаемая дата выдачи РД в производство_new'] = changed_df['Ожидаемая дата выдачи РД в производство_new'].apply(
#     lambda row: row if row == 'в производстве' or row == 'В производстве' else pd.to_datetime(row).date()
#     )
# changed_df.columns = new_columns

# Дополнение базы разностью дат
# Не удовлетворяют условиям
# tm_df_1 = changed_df[(pd.isnull(changed_df['Ожидаемая дата выдачи РД в производство_new'])) | 
#            (pd.isnull(changed_df['Дата выпуска РД по договору подрядчика_old'])) |
#            (changed_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) |
#            (changed_df['Дата выпуска РД по договору подрядчика_old'].isin(['в производстве', 'В производстве']))]
# # Удовлетворяют условиям
# tm_df_2 = changed_df[(~pd.isnull(changed_df['Ожидаемая дата выдачи РД в производство_new'])) & 
#            (~pd.isnull(changed_df['Дата выпуска РД по договору подрядчика_old'])) &
#            (~changed_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) &
#            (~changed_df['Дата выпуска РД по договору подрядчика_old'].isin(['в производстве', 'В производстве']))]
# tm_df_2['Задержка договора'] = (tm_df_2['Ожидаемая дата выдачи РД в производство_new'] - tm_df_2['Дата выпуска РД по договору подрядчика_old']).dt.days
# tm_df_1['Задержка договора'] = tm_df_1['Дата выпуска РД по договору подрядчика_old']
# tm_df = pd.concat([tm_df_1, tm_df_2])
# # Удовлетворяют условиям
# tm_df_1 = tm_df[(~pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_new'])) & 
#            (~pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_old'])) &
#            (~tm_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) &
#            (~tm_df['Ожидаемая дата выдачи РД в производство_old'].isin(['в производстве', 'В производстве']))]
# tm_df_1['Разница ожидаемых дат new-old'] = (tm_df_1['Ожидаемая дата выдачи РД в производство_new'] - tm_df_1['Ожидаемая дата выдачи РД в производство_old']).dt.days
# # Не удовлетворяют условиям
# tm_df_2 = tm_df[(pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_new'])) |
#            (pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_old'])) |
#            (tm_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) |
#            (tm_df['Ожидаемая дата выдачи РД в производство_old'].isin(['в производстве', 'В производстве']))]
# tm_df_2['Разница ожидаемых дат new-old'] = tm_df_2['Ожидаемая дата выдачи РД в производство_new']
# tm_df_2['Разница ожидаемых дат new-old'] = tm_df_2['Разница ожидаемых дат new-old'].apply(
#     lambda row: tm_df_2['Ожидаемая дата выдачи РД в производство_old'] if row is None else row
# )
# tm_df = pd.concat([tm_df_1, tm_df_2])
# tm_df.to_excel('dates.xlsx', index = False)

# comf_ren = input('Use standard file name (y/n): ')
# while comf_ren not in 'YyNn':
#     comf_ren = input('For next work choose <y> or <n> simbols): ')

# if comf_ren in 'Yy':
#     output_filename = 'result' + str(datetime.now().isoformat(timespec='minutes')).replace(':', '_')
# else:
#     output_filename = input('Input result file name: ')
# changed_df.to_excel(f'./{output_filename}.xlsx', encoding='cp1251', index = False)

# # Поправить данные в этой части, рассмотреть дату и время
# comf_ren = input('Use standard file name for log file (y/n): ')
# while comf_ren not in 'YyNn':
#     comf_ren = input('For next work choose <y> or <n> simbols): ')

# if comf_ren in 'Yy':
#    output_filename = 'log-RD-' + str(datetime.now().isoformat(timespec='minutes')).replace(':', '_')
# else:
#     output_filename = input('Input result file name: ')
# m_df_2[m_df_2['_merge'] == 'both'].to_excel(f'./{output_filename}.xlsx', encoding='cp1251', index = False)

Read excel files with current and new data
Finding missed rows
Clear the empty rows in both dataframes
Clear the unnecessary data in base dataframe
Making copy of original dataframes
Merging two dataframes
Initiate dataframe with changed documents
Generating result dataframe, with remove document firstly
Generating result dataframe 
Changing dataframe


In [20]:
columns = ['Система',
           'Наименование объекта/комплекта РД',
           'Коды работ по выпуску РД',
           'Ожидаемая дата выдачи РД в производство',
           'Дата выпуска РД по договору подрядчика',
           'Дата выпуска РД по графику с Заказчиком',
           'Ожидаемая дата выдачи РД в производство_new',
           'Дата выпуска РД по договору подрядчика_new',
           'Дата выпуска РД по графику с Заказчиком_new']
new_columns = ['Система',
               'Наименование объекта/комплекта РД',
               'Коды работ по выпуску РД',
               'Ожидаемая дата выдачи РД в производство_old',
               'Дата выпуска РД по договору подрядчика_old',
               'Дата выпуска РД по графику с Заказчиком_old',
               'Ожидаемая дата выдачи РД в производство_new',
               'Дата выпуска РД по договору подрядчика_new',
               'Дата выпуска РД по графику с Заказчиком_new']
changed_df = pd.concat([m_df_1[m_df_1['_merge'] == 'both'][columns], m_df_2[m_df_2['_merge'] == 'left_only'][columns]])
changed_df = pd.concat([changed_df, m_df_2[m_df_2['_merge'] == 'both'][columns]])
changed_df =  changed_df[~changed_df['Система'].isin(['ГПМ', 'ИМ', "КИТС ФЗ", "МиЗ", "ОРС", "РЗА", "Связь", "СКУ ЭЧ ОС СН", "СКУ ЭЧ ЭБ", "Технология"])]
changed_df['Дата выпуска РД по договору подрядчика'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика']).dt.date
changed_df['Дата выпуска РД по графику с Заказчиком'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком']).dt.date
changed_df['Дата выпуска РД по договору подрядчика_new'] = pd.to_datetime(changed_df['Дата выпуска РД по договору подрядчика_new']).dt.date
changed_df['Дата выпуска РД по графику с Заказчиком_new'] = pd.to_datetime(changed_df['Дата выпуска РД по графику с Заказчиком_new']).dt.date
changed_df['Ожидаемая дата выдачи РД в производство'] = changed_df['Ожидаемая дата выдачи РД в производство'].apply(
    lambda row: row if row == 'в производстве' or row == 'В производстве' else pd.to_datetime(row).date()
    )
changed_df['Ожидаемая дата выдачи РД в производство_new'] = changed_df['Ожидаемая дата выдачи РД в производство_new'].apply(
    lambda row: row if row == 'в производстве' or row == 'В производстве' else pd.to_datetime(row).date()
    )
changed_df.columns = new_columns

changed_df['Задержка договора'] = changed_df['Дата выпуска РД по договору подрядчика_old']
# tm_df['Задержка договора'] = tm_df['Задержка договора'].apply(
#     lambda x: 1 if x is None and x in ('в производстве', 'В производстве') 
#     else tm_df['Ожидаемая дата выдачи РД в производство_new'].apply(
#                 lambda y: x if y is None and y in ('в производстве', 'В производстве') 
#                 else y-x)
# )
changed_df[(~pd.isnull(changed_df['Ожидаемая дата выдачи РД в производство_new'])) & 
           (~pd.isnull(changed_df['Дата выпуска РД по договору подрядчика_old'])) &
           (~changed_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) &
           (~changed_df['Дата выпуска РД по договору подрядчика_old'].isin(['в производстве', 'В производстве']))]

In [208]:

from copy import deepcopy

# warnings.simplefilter(action='ignore', category=(FutureWarning, UserWarning))

# Не удовлетворяют условиям
tm_df_1 = changed_df[(pd.isnull(changed_df['Ожидаемая дата выдачи РД в производство_new'])) | 
           (pd.isnull(changed_df['Дата выпуска РД по договору подрядчика_old'])) |
           (changed_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) |
           (changed_df['Дата выпуска РД по договору подрядчика_old'].isin(['в производстве', 'В производстве']))]
# Удовлетворяют условиям
tm_df_2 = changed_df[(~pd.isnull(changed_df['Ожидаемая дата выдачи РД в производство_new'])) & 
           (~pd.isnull(changed_df['Дата выпуска РД по договору подрядчика_old'])) &
           (~changed_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) &
           (~changed_df['Дата выпуска РД по договору подрядчика_old'].isin(['в производстве', 'В производстве']))]
t_df1_copy = tm_df_1.copy()
t_df2_copy = tm_df_2.copy()
t_df1_copy['Задержка договора'] = t_df1_copy['Дата выпуска РД по договору подрядчика_old']
t_df2_copy['Задержка договора'] = (t_df2_copy['Ожидаемая дата выдачи РД в производство_new'] - t_df2_copy['Дата выпуска РД по договору подрядчика_old']).dt.days
tm_df = pd.concat([t_df1_copy, t_df2_copy])

# Удовлетворяют условиям
tm_df_1 = tm_df[(~pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_new'])) & 
           (~pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_old'])) &
           (~tm_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) &
           (~tm_df['Ожидаемая дата выдачи РД в производство_old'].isin(['в производстве', 'В производстве']))]
# Не удовлетворяют условиям
tm_df_2 = tm_df[(pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_new'])) |
           (pd.isnull(tm_df['Ожидаемая дата выдачи РД в производство_old'])) |
           (tm_df['Ожидаемая дата выдачи РД в производство_new'].isin(['в производстве', 'В производстве'])) |
           (tm_df['Ожидаемая дата выдачи РД в производство_old'].isin(['в производстве', 'В производстве']))]
t_df1_copy = tm_df_1.copy()
t_df2_copy = tm_df_2.copy()
t_df1_copy['Разница ожидаемых дат new-old'] = (t_df1_copy['Ожидаемая дата выдачи РД в производство_new'] - t_df1_copy['Ожидаемая дата выдачи РД в производство_old']).dt.days
td1 = t_df2_copy.loc[pd.isnull(t_df2_copy['Ожидаемая дата выдачи РД в производство_new']) | pd.isnull(t_df2_copy['Ожидаемая дата выдачи РД в производство_old'])]
td2 = t_df2_copy.loc[~pd.isnull(t_df2_copy['Ожидаемая дата выдачи РД в производство_new']) & ~pd.isnull(t_df2_copy['Ожидаемая дата выдачи РД в производство_old'])]
td1_copy = td1.copy()
td2_copy = td2.copy()
td1_copy['Разница ожидаемых дат new-old'] = pd.NaT
td2_copy['Разница ожидаемых дат new-old'] = 'в производстве'
tdf = pd.concat([td1_copy, td2_copy])
tdf = pd.concat([t_df1_copy, tdf])
tdf.sort_index().to_excel('1.xlsx')


In [211]:

df_1 = tdf.loc[tdf['Задержка договора'].apply(lambda row: not isinstance(row, int) and row is not pd.NaT)]
df_2 = tdf.loc[tdf['Задержка договора'].apply(lambda row: isinstance(row, int) or row is  pd.NaT)]
# cdf_1 = df_1.copy()
# cdf_2 = df_2.copy()
# cdf_1['Задержка договора'] = 'в производстве'
# df = pd.concat([cdf_1, cdf_2])
# df_1 = df.loc[df['Разница ожидаемых дат new-old'].apply(lambda row: not isinstance(row, int) and row is not pd.NaT)]
# df_2 = df.loc[df['Разница ожидаемых дат new-old'].apply(lambda row: isinstance(row, int) or row is  pd.NaT)]
# cdf_1 = df_1.copy()
# cdf_2 = df_2.copy()
# cdf_1['Разница ожидаемых дат new-old'] = 'в производстве'
# df = pd.concat([cdf_1, cdf_2])
# df.sort_index().to_excel('Dates(new).xlsx', index = False)
df_2

,Система,Наименование объекта/комплекта РД,Коды работ по выпуску РД,Ожидаемая дата выдачи РД в производство_old,Дата выпуска РД по договору подрядчика_old,Дата выпуска РД по графику с Заказчиком_old,Ожидаемая дата выдачи РД в производство_new,Дата выпуска РД по договору подрядчика_new,Дата выпуска РД по графику с Заказчиком_new,Задержка договора,Разница ожидаемых дат new-old
287,КИП,Система нейтрализации сбросных вод после химич...,00UGD.J.GNR.D-1000,2022-12-12,NaT,NaT,2023-03-02,2022-12-31,NaT,NaT,80
524,СКУ ПЗ,Автоматические установки пожаротушения,00UKU.T.000.D-1110,2022-11-20,NaT,NaT,2023-03-02,2022-11-30,NaT,NaT,102
806,СКУ МПУ ОС,СКУ МПУ ОС. Задание заводу на ПТК. 2 этап,00UXX.E.EEE.D-1010,2023-01-25,NaT,NaT,2023-05-25,NaT,NaT,NaT,120
812,СКУ ПЗ,СКУ ПЗ. Задание заводу на ПТК. 1 этап,00UXX.J.JJJ.D-1050,2023-01-29,NaT,NaT,2023-03-02,NaT,NaT,NaT,32
813,СКУ ПЗ,СКУ ПЗ. Задание заводу на ПТК. 2 этап,00UXX.J.JJJ.D-1060,2022-12-31,NaT,NaT,2023-02-16,NaT,NaT,NaT,47
...,...,...,...,...,...,...,...,...,...,...,...
3889,СКУ ЭЧ ОС,Силовое электрооборудование,52UGH.E.B00.D-1040,в производстве,NaT,NaT,в производстве,NaT,NaT,NaT,в производстве
3893,СКУ ЭЧ ОС,Силовое электрооборудование,53UGH.E.B00.D-1040,в производстве,NaT,NaT,в производстве,NaT,NaT,NaT,в производстве
3895,СКУ ЭЧ ОС,Технологическое задание на изготовление блочно...,53UGJ.E.B00.D-1020,в производстве,NaT,NaT,в производстве,NaT,NaT,NaT,в производстве
3896,СКУ ЭЧ ОС,Технологическое задание на изготовление распре...,53UGJ.E.B00.D-1030,в производстве,NaT,NaT,в производстве,NaT,NaT,NaT,в производстве
